# Requirements

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import clear_output


pd.set_option('mode.chained_assignment', None)

## timeline strategy

### load database

In [2]:
database_dir_path = "D:\Projects\Rich\Cmong\Stock\database\my"

In [3]:
df_list = []
df_name_list = []
df_fluc_list = []

for p_i, pkl in enumerate(os.listdir(database_dir_path)):
    
    print(p_i)
    df = pd.read_excel(os.path.join(database_dir_path, pkl), index_col=0)
    # display(df.head())
    # break
    
    try:
        old_cols = ['시가', '고가', '저가', '현재가', '거래량']
        df = df[old_cols]
        df.columns = ['open', 'high', 'low', 'close', 'volume']
    except:
        pass
    
    price_fluctuation = df.close / df.close.shift(1)  
    price_fluctuation.name = p_i
    
    df_list.append(df)
    df_name_list.append(pkl)
    df_fluc_list.append(price_fluctuation)
    
    if p_i > 20:
        break
    
                  
    clear_output(wait=True)

21


### get top rank

In [4]:
df_flucs_rank = pd.concat(df_fluc_list, axis=1).rank(ascending=False, method='first',  axis=1)

top_rank = 10
df_buy = df_flucs_rank <= top_rank

### adj. strategy

In [ ]:
df_fluc_list

In [15]:
take_profit = 1.1  # 10%
stop_loss = 0.95   # -5%

fees = 0.00015 + 0.00015  # 0.015% 수수료

save_path = database_dir_path.replace('database', 'result')


buys  = []
holds = []
sells = []
profits = []
profit_cums = []


col_i = 0  # symbol_index, for test.
for col_i in range(len(df_fluc_list)):

    en_ex_total = []
    en_ex = []

    for row, signal in zip(df_list[col_i].iterrows(), df_buy.iloc[:, col_i]):

        en_ex_len = len(en_ex)
        

        # 1. check buy signal.
            # a. 중복 매수는 허용하지 않는다.
        if signal and en_ex_len == 0:
            en_ex.append(row[-1].name)
            en_ex.append(row[-1]['close'])
            
        # 2. check sell signal.
        if en_ex_len > 0:

            exit_done = 0

            # a. take_profit
            if row[-1]['high'] / en_ex[1] >= take_profit:
                profit = take_profit * (1 - fees - 0.0018) # 증권거래세 등(0.18%)
                exit_done = 1
            elif row[-1]['low'] / en_ex[1] <= stop_loss:
                profit = stop_loss * (1 - fees)
                exit_done = -1

            if exit_done:
                en_ex.append(row[-1].name)
                en_ex.append(exit_done)
                en_ex.append(profit)
                en_ex_total.append(en_ex)
                en_ex = []
            # else:
            #     en_ex.append(row[-1].name)
            #     en_ex.append(exit_done)
            #     en_ex.append(np.nan)
            #     en_ex_total.append(en_ex)
                


    en_ex_total_arr = np.array(en_ex_total, dtype=object)

    # 3. initialization
    df_list[col_i]['buy'] = np.nan
    df_list[col_i]['sell'] = np.nan
    df_list[col_i]['profit'] = np.nan
    df_list[col_i]['profit_cum'] = 1

    # 4. enlist trade info.
    df_list[col_i]['buy'].loc[en_ex_total_arr[:, 0]] = 1
    df_list[col_i]['sell'].loc[en_ex_total_arr[:, 2]] = en_ex_total_arr[:, 3]
    df_list[col_i]['profit'].loc[en_ex_total_arr[:, 2]] = en_ex_total_arr[:, 4]
    df_list[col_i]['profit_cum'].loc[en_ex_total_arr[:, 2]] = en_ex_total_arr[:, 4]
    df_list[col_i]['profit_cum'] = np.cumprod(df_list[col_i]['profit_cum'])

    # display(df_list[col_i])

    # plt.figure(figsize=(5,5))
    # plt.plot(df_list[col_i]['profit_cum'])
    # plt.title(df_name_list[col_i])
    # plt.show()
    
    df_list[col_i].to_excel(os.path.join(save_path, '{}'.format(df_name_list[col_i])), index=False)
    
#     buys.append(df_list[col_i]['buy'].to_numpy())
#     holds.append(df_list[col_i]['sell'].to_numpy())
#     sells.append(df_list[col_i]['sell'].to_numpy())
#     profits.append(df_list[col_i]['profit'].to_numpy())
#     profit_cums.append(df_list[col_i]['profit_cum'].to_numpy())
    
    break

In [16]:

print(en_ex_total_arr[0])

[Timestamp('2018-12-19 15:30:00') 37350.0 Timestamp('2019-01-03 15:30:00')
 -1 0.949715]


## backtrader

In [6]:
from datetime import datetime
import backtrader as bt

# Create a subclass of Strategy to define the indicators and logic

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position


cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

# Create a data feed
# datalist.append(df)
data = bt.feeds.PandasData(dataname=df)

cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(SmaCross)  # Add the trading strategy
cerebro.run()  # run it all

cerebro.plot(iplot=False)  # and plot it with a single command


[[<Figure size 640x480 with 5 Axes>]]

In [ ]:
# df# !pip list